In [26]:
from pydantic import BaseModel
from typing import Literal, Any, Dict, Optional, List

class OpenAIParameterProperty(BaseModel):
    type: str
    description: str
    enum: Optional[List[str]] = None
    properties: Optional[Dict[str, Dict[str, Any]]] = None

class OpenAIParameters(BaseModel):
    type: Literal["object"]
    properties: Dict[str, Dict[str, Any]]
    required: Optional[List[str]]

class OpenAIFunction(BaseModel):
    name: str
    description: str
    parameters: OpenAIParameters

class OpenAITool(BaseModel):
    type: Literal["function"]
    function: OpenAIFunction

def build_weaviate_query_tool_for_openai(collections_description: str, collections_list: list[str], generate_with_models: bool = False) -> OpenAITool:

    if generate_with_models:
        properties = {
            "collection_name": {
                "type": "string",
                "description": "The collection to query. Has a boolean-valued property `inBoston` that you can use to filter queries on if useful.",
                "enum": collections_list
            },
            "search_query": {
                "type": "string",
                "description": "A search query to return objects from a search index."
            },
            "integer_property_filter": {
                "type": "object",
                "description": "Filter numeric properties using comparison operators",
                "properties": {
                    "property_name": {"type": "string"},
                    "operator": {"type": "string", "enum": ["=", "<", ">", "<=", ">="]},
                    "value": {"type": "number"}
                }
            },
            "text_property_filter": {
                "type": "object", 
                "description": "Filter text properties using equality or LIKE operators",
                "properties": {
                    "property_name": {"type": "string"},
                    "operator": {"type": "string", "enum": ["=", "LIKE"]},
                    "value": {"type": "string"}
                }
            },
            "boolean_property_filter": {
                "type": "object",
                "description": "Filter boolean properties using equality operators",
                "properties": {
                    "property_name": {"type": "string"},
                    "operator": {"type": "string", "enum": ["=", "!="]},
                    "value": {"type": "boolean"}
                }
            },
            "integer_property_aggregation": {
                "type": "object",
                "description": "Aggregate numeric properties using statistical functions",
                "properties": {
                    "property_name": {"type": "string"},
                    "metrics": {"type": "string", "enum": ["COUNT", "TYPE", "MIN", "MAX", "MEAN", "MEDIAN", "MODE", "SUM"]}
                }
            },
            "text_property_aggregation": {
                "type": "object",
                "description": "Aggregate text properties using frequency analysis",
                "properties": {
                    "property_name": {"type": "string"},
                    "metrics": {"type": "string", "enum": ["COUNT", "TYPE", "TOP_OCCURRENCES"]},
                    "top_occurrences_limit": {"type": "integer"}
                }
            },
            "boolean_property_aggregation": {
                "type": "object",
                "description": "Aggregate boolean properties using statistical functions",
                "properties": {
                    "property_name": {"type": "string"},
                    "metrics": {"type": "string", "enum": ["COUNT", "TYPE", "TOTAL_TRUE", "TOTAL_FALSE", "PERCENTAGE_TRUE", "PERCENTAGE_FALSE"]}
                }
            },
            "groupby_property": {
                "type": "string",
                "description": "Group the results by a property."
            }
        }
    else:
        properties = {
            "collection_name": {
                "type": "string",
                "description": "A collection to search through.",
                "enum": collections_list
            },
            "search_query": {
                "type": "string",
                "description": "A search query"
            },
            "filter_string": {
                "type": "string",
                "description": "A filter to apply to the search results."
            },
            "aggregate_string": {
                "type": "string",
                "description": "An aggregation to apply to the search results."
            }
        }

    return OpenAITool(
        type="function",
        function=OpenAIFunction(
            name="query_database",
            description=f"""Query a database.

            Available collections in this database:
            {collections_description}""",
            parameters=OpenAIParameters(
                type="object",
                properties=properties,
                required=["collection_name"]
            )
        )
    )

In [27]:
tools = [build_weaviate_query_tool_for_openai(
    "test",
    ["a", "b", "c"],
    True
).model_dump()]

In [28]:
import openai

openai_client = openai.OpenAI(
    api_key = ""
)

prompt = "Show me restaurants in Boston"

messages = [
    {"role": "system", "content": "You are a helpful assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": prompt}
]

response = openai_client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=messages,
    tools=tools,
)

In [29]:
print(response.choices[0].message.tool_calls[0].function.arguments)


{"collection_name":"b","boolean_property_filter":{"property_name":"inBoston","operator":"=","value":true}}
